In [13]:
import pandas as pd

In [14]:
df1 = pd.read_csv("results/tumor_22wk.txt", sep="\t", header=None)
df2 = pd.read_csv("results/control_22wk.txt", sep="\t", header=None)

/projectnb/bf528/students/jgsherry/.conda/envs/notebook2_env/lib/python3.6/site-packages/IPython/core/interactiveshell.py:3072: DtypeWarning: Columns (1) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [15]:
def rm_cols(df):
    cols_to_add = [0,1,2,3]
    for i in range(5, df.shape[1], 1):
        if i % 2 != 0:
            cols_to_add.append(i)
    return df[cols_to_add]

df1 = rm_cols(df1)
df2 = rm_cols(df2)

In [16]:
df1 = df1[~(df1.eq('.').all(axis=1))]

In [17]:
def fix_header(df):
    df.columns = df.iloc[0]   # Set first row as header
    df = df[1:]               # Drop the first row (now header)
    return df[1:].reset_index(drop=True)
df1 = fix_header(df1)
df2 = fix_header(df2)

In [18]:
def get_num(col_values):
    num = []
    for item in col_values:
        if not item or item in ('.', './.'):
            num.append(0)
        else:
            parts = item.split(',')
            numbers = [int(x) if x != '.' else 0 for x in parts]
            num.append(sum(numbers[1:]) if len(numbers) > 1 else 0)
    return num

def get_denom(col_values):
    denom = []
    for item in col_values:
        if not item or item in ('.', './.'):
            denom.append(0)
        else:
            parts = item.split(',')
            numbers = [int(x) if x != '.' else 0 for x in parts]
            denom.append(sum(numbers))
    return denom

def sorting(df):
    nums = []
    denoms = []

    # Select all AD columns
    ad_cols = [c for c in df.columns if c.startswith('AD_')]
    # Select all DP columns (for denominators)
    dp_cols = [c for c in df.columns if c.startswith('DP_')]

    for ad_col, dp_col in zip(ad_cols, dp_cols):
        ad_list = df[ad_col].tolist()
        dp_list = df[dp_col].tolist()

        nums.append(get_num(ad_list))
        denoms.append(get_denom(dp_list))

    # Sum across columns
    nums_col_sum = [sum(col) for col in zip(*nums)]
    denoms_col_sum = [sum(col) for col in zip(*denoms)]

    ratios = [n/d if d != 0 else 0 for n, d in zip(nums_col_sum, denoms_col_sum)]

    df["snp_rate"] = ratios
    df["snp_cnt"] = denoms_col_sum

    final_cols_to_keep = ['CHROM', 'POS', 'REF', 'ALT', 'snp_rate', 'snp_cnt']
    return df[final_cols_to_keep]
    
df1 = sorting(df1)
df2 = sorting(df2)

In [19]:
dfm = pd.merge(df1, df2, on=['CHROM','POS'], how='left')

In [20]:
dfm = dfm.fillna(0)

In [25]:
dfm = dfm[(dfm['snp_cnt_x'] > 30) & (dfm['snp_rate_x'] > 0.5) & (dfm['snp_rate_y'] < 0.2) & (dfm['snp_cnt_y'] > 0)]
dfm

,CHROM,POS,REF_x,ALT_x,snp_rate_x,snp_cnt_x,REF_y,ALT_y,snp_rate_y,snp_cnt_y
193,chr10,21116033,GACAACA,"AGCAGCG,G",0.941176,85,GACAACA,G,0.156863,255.0
3710,chr11,80286377,CCCCTCCCT,C,0.574468,47,CCCCTCCCTCCCTCCCT,C,0.166667,12.0
6445,chr12,104970411,G,A,0.578730,1213,G,A,0.182017,4571.0
6514,chr12,105092956,C,T,0.619500,13251,C,T,0.169074,3046.0
9682,chr14,55517002,TATTT,"TATTTA,ATATTT",0.523256,86,TATTT,ATATTTA,0.190476,21.0
10527,chr15,10276610,CCT,"C,CCTCTCT,CCTCT",0.525000,120,C,"CCTCT,CCT",0.162304,191.0
19024,chr19,40172456,G,A,0.517746,10735,G,A,0.151163,172.0
30356,chr5,28404383,A,G,0.959184,49,A,G,0.142433,337.0
30710,chr5,66408867,GAAAA,"GAA,G,GA",0.567568,37,GAA,"GAAA,G",0.163636,55.0
35589,chr7,30946563,A,G,0.509434,53,A,G,0.190476,21.0


In [22]:
#dfm[(dfm['snp_cnt_x'] > 30) & (dfm['snp_rate_x'] > 0.5) & (dfm['snp_rate_y'] < 0.2) & (dfm['snp_cnt_y'] > 0)]

Now for mapping to gene names

In [26]:

genes_df = pd.read_csv("mm9_filtered_gene_locs.tsv", sep="\t")

# Ensure chromosome column matches your mutations df
genes_df['chrom'] = genes_df['chrom'].astype(str)
genes_df

,name,chrom,txStart,txEnd
0,NM_028778,chr1,134212701,134230065
1,NM_001195025,chr1,134212701,134230065
2,NM_001347056,chr1,58714964,58749289
3,NM_175370,chr1,58714964,58752833
4,NM_001290393,chr1,8349819,9289958
...,...,...,...,...
47242,NM_009488,chrUn_random,2735817,2743632
47243,NM_001033769,chrUn_random,2889606,2891056
47244,NM_029975,chrUn_random,3058602,3072666
47245,NR_126081,chrUn_random,4834599,4838176


In [27]:

# Load your gene coordinates
genes_df = pd.read_csv("mm9_filtered_gene_locs.tsv", sep="\t")

# Make sure chromosome columns are strings
genes_df['chrom'] = genes_df['chrom'].astype(str)
dfm['CHROM'] = dfm['CHROM'].astype(str)

# Convert positions to integers
dfm['POS'] = pd.to_numeric(dfm['POS'], errors='coerce')
genes_df['txStart'] = pd.to_numeric(genes_df['txStart'], errors='coerce')
genes_df['txEnd'] = pd.to_numeric(genes_df['txEnd'], errors='coerce')

# Drop rows with NaN positions (optional)
dfm = dfm.dropna(subset=['POS'])
genes_df = genes_df.dropna(subset=['txStart','txEnd'])

# Merge by chromosome
merged = pd.merge(dfm, genes_df, left_on='CHROM', right_on='chrom', how='left')

# Keep only rows where POS falls within gene coordinates
mapped = merged[(merged['POS'] >= merged['txStart']) & (merged['POS'] <= merged['txEnd'])]

# Get unique genes
genes_seen = mapped['name'].unique()

'''
# Save to a text file
with open("genes_seen.txt", "w") as f:
    for gene in genes_seen:
        f.write(gene + "\n")
'''
genes_seen

/projectnb/bf528/students/jgsherry/.conda/envs/notebook2_env/lib/python3.6/site-packages/ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
/projectnb/bf528/students/jgsherry/.conda/envs/notebook2_env/lib/python3.6/site-packages/ipykernel_launcher.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  if __name__ == '__main__':


array(['NM_178713', 'NM_009244', 'NM_001252569', 'NM_009243', 'NM_019402',
       'NM_001253781', 'NM_001167875', 'NM_134144', 'NM_153526',
       'NM_001127382', 'NM_001291226', 'NM_139065', 'NM_178446',
       'NM_001357557', 'NM_011980', 'NM_028166', 'NM_001085385'],
      dtype=object)